# Quantro Heart Model - Getting Started

Welcome to the Quantro Heart Model simulation framework. This notebook provides an introduction to the models and basic usage.

## Overview

This framework implements five cardiac-relevant models:

1. **Michaelis-Menten (MM)**: Enzyme kinetics for cardiac metabolism
2. **SIR Model**: Compartmental dynamics for activation spread
3. **FitzHugh-Nagumo (FHN)**: Cardiac action potential generation
4. **Nernst Potential**: Ion concentration and membrane potentials
5. **Poiseuille Flow**: Blood flow hemodynamics

## Installation

Ensure you have the required packages installed:

```bash
pip install numpy pandas matplotlib scipy jupyter
```

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

# Add parent directory to path
sys.path.insert(0, '..')

from quantro_simulator import (
    SimulationConfig,
    ModelType,
    OverlayMode,
    create_model,
    RK4Integrator,
    run_parameter_sweep
)

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## Example 1: Running a Single Simulation

Let's run a simple SIR model simulation:

In [ ]:
# Configure simulation
config = SimulationConfig(
    model=ModelType.SIR,
    overlay_mode=OverlayMode.BASELINE,
    t_start=0.0,
    t_end=20.0,
    dt=0.01
)

# Create and run model
model = create_model(config)
time_points, trajectory = RK4Integrator.integrate(model, lambda_param=0.0)

# Plot results
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(time_points, trajectory[:, 0], label='Susceptible (S)', linewidth=2)
ax.plot(time_points, trajectory[:, 1], label='Infected (I)', linewidth=2)
ax.plot(time_points, trajectory[:, 2], label='Recovered (R)', linewidth=2)
ax.set_xlabel('Time (arbitrary units)', fontsize=12)
ax.set_ylabel('Population Fraction', fontsize=12)
ax.set_title('SIR Model: Disease Spread Dynamics', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Final state: S={trajectory[-1, 0]:.4f}, I={trajectory[-1, 1]:.4f}, R={trajectory[-1, 2]:.4f}")

## Example 2: Comparing Overlay Modes

Let's see how different overlay modes affect the FitzHugh-Nagumo model:

In [ ]:
overlay_modes = [OverlayMode.BASELINE, OverlayMode.RESIDUAL, OverlayMode.CONTROL]

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, mode in enumerate(overlay_modes):
    config = SimulationConfig(
        model=ModelType.FITZHUGH_NAGUMO,
        overlay_mode=mode,
        t_start=0.0,
        t_end=50.0,
        dt=0.01
    )
    
    model = create_model(config)
    time_points, trajectory = RK4Integrator.integrate(model, lambda_param=0.5)
    
    axes[i].plot(time_points, trajectory[:, 0], 'b-', linewidth=1.5, label='Membrane Potential (v)')
    axes[i].plot(time_points, trajectory[:, 1], 'r--', linewidth=1.5, label='Recovery (w)')
    axes[i].set_xlabel('Time', fontsize=10)
    axes[i].set_ylabel('State Variables', fontsize=10)
    axes[i].set_title(f'{mode.value} Mode', fontsize=11, fontweight='bold')
    axes[i].legend(fontsize=9)
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Example 3: Parameter Sweep

Explore how the lambda parameter affects model behavior:

In [ ]:
config = SimulationConfig(
    model=ModelType.MICHAELIS_MENTEN,
    overlay_mode=OverlayMode.PARAM_MOD,
    t_start=0.0,
    t_end=10.0,
    dt=0.01,
    lambda_values=np.linspace(0.0, 1.0, 20)
)

results = run_parameter_sweep(config)

# Extract data
lambda_vals = [r.lambda_param for r in results]
val1_vals = [r.val1 for r in results]
val3_vals = [r.val3 for r in results]

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(lambda_vals, val1_vals, 'o-', linewidth=2, markersize=6)
ax1.set_xlabel('Lambda Parameter', fontsize=11)
ax1.set_ylabel('Mean Substrate Concentration', fontsize=11)
ax1.set_title('Average State vs Lambda', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3)

ax2.plot(lambda_vals, val3_vals, 's-', color='orangered', linewidth=2, markersize=6)
ax2.set_xlabel('Lambda Parameter', fontsize=11)
ax2.set_ylabel('Final State Value', fontsize=11)
ax2.set_title('Final State vs Lambda', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Next Steps

Explore the other notebooks:

- **02_Parameter_Sweep_Analysis**: Deep dive into parameter space exploration
- **03_FitzHugh_Nagumo_Analysis**: Cardiac action potential dynamics
- **04_Hemodynamics_Analysis**: Blood flow and vessel mechanics
- **05_Comparative_Analysis**: Cross-model comparisons and insights

## HIPAA Compliance Note

⚠️ **Important**: This framework includes HIPAA-compliant data handling features. Ensure you:
- Use encrypted storage for patient data
- Enable audit logging when processing PHI
- Follow organizational security protocols

See `security_config.py` for configuration details.